# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré. Puisque dans ce POC nous utilisons YoloV5, il faut installer de nombreuses dépendances dont PyTorch.

<div class="alert alert-info">
    Afin d'exécuter ce Notebook sur AWS SageMaker, il faut utiliser le kernel <code>conda_python3</code>.
</div>

<div class="alert alert-info">
    Il faut compter environ 5 minutes pour l'installation des dépendances.
</div>

In [1]:
!git clone https://github.com/rkuo2000/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12560, done.
remote: Total 12560 (delta 0), reused 0 (delta 0), pack-reused 12560
Receiving objects: 100% (12560/12560), 12.37 MiB | 26.61 MiB/s, done.
Resolving deltas: 100% (8578/8578), done.


On installe ensuite les dépendances pour YoloV5.

In [2]:
!pip install -q -r yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00


Enfin, nous installons les dépendances liées aux frameworks Deep Learning (ici PyTorch).

<div class="alert alert-warning">
    Attention, il faut utiliser la bonne version de PyTorch : sur SageMaker, c'est la version <code>1.11.0+cu113</code> mais sur un autre environment, cela peut être différent !
</div>

In [3]:
!pip install -q \
    torch==1.11.0+cu113 \
    torchvision==0.12.0+cu113 \
    torchaudio==0.11.0 \
    gcc7 \
    opendatasets \
    pycocotools \
    split-folders \
    --extra-index-url https://download.pytorch.org/whl/cu113

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 969.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 MB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.11.0+cu113 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.11.0+cu113 which is incompatible.


In [4]:
# Ceci est une petite correction nécessaire pour YoloV5
!pip install -q --upgrade scipy

<div class="alert alert-info">
    Il est nécessaire de créer un compte sur <a href="https://kaggle.com" target="_blank">Kaggle</a> pour télécharger les données.
</div>

In [5]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/kneroma/tacotrashdataset")

100%|██████████| 2.79G/2.79G [00:23<00:00, 125MB/s]


# Préparation des données

Le but de cette partie est de **transformer le fichier d'annotations JSON en plusieurs fichier texte** que le modèle YoloV5 va utiliser pour s'entraîner et notamment apprendre quelles bounding boxes il doit être capable de retrouver.

Dans ce format, il y a ainsi **un fichier texte par image** où pour chaque déchet dans l'image est associée une ligne contenant l'identifiant du déchet et les coordonnées de la bounding box. Dans l'exemple suivant, l'image associée contiendrait 2 types de déchets (identifiants 4 et 7) et 3 déchets présents sur l'image.

In [6]:
from pycocotools.coco import COCO

# On importe le fichier des annotations sous la forme d'un objet COCO
data_source = COCO(annotation_file="./tacotrashdataset/data/annotations.json")

loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [ ]:
data_source

On extrait ensuite les identifiants des images, des catégories et des super catégories.

### ➡️ À toi de jouer

Créer les objets Python suivants.

- `categories` est une lmiste contient l'ensemble des catégories avec un identifiant (`id`), le nom de la catégorie (`name`) et le nom de la super catégorie (`supecategory`). Par exemple, on aura `[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'}, ...]`.
- `classes_num` est un dictionnaire qui indique l'identifiant (`id`) de n'importe quel catégorie. Par exemple, on aura `{'Aluminium foil': 0, 'Battery': 1, ...}`.
- `coco_labels` est un dictionnaire qui contient, pour chaque clé incrémentale, l'identifiant de la catégorie. Par exemple, on aura `{0: 0, 1: 1, ...}`. On utilise ce dictionnaire si par exemple on ne souhaite pas utiliser 100% des catégories disponibles, mais uniquement une partie.
- `coco_labels_inverse` est le dictionnaire inverse de `coco_labels` : si `coco_labels[A] = B`, alors `coco_labels_inverse[B] = A`.

In [11]:
img_ids = data_source.getImgIds()
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
# Trier les catégories selon l'ID
# TODO
categories = sorted(categories, key  = lambda id : id["id"])
print(categories)

classes = {}
coco_labels = {}
coco_labels_inverse = {}
class_num = {}
i = 0
# Pour chaque catégorie
for c in categories:

    # Remplir les variables coco_labels, coco_labels_inverse et classes
    # TODO

    coco_labels[len(classes)] = c['id']
    coco_labels_inverse[c['id']] = len(classes)
    classes[c['name']] = len(classes)
    class_num[c['name']] = c['id']

[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'}, {'supercategory': 'Battery', 'id': 1, 'name': 'Battery'}, {'supercategory': 'Blister pack', 'id': 2, 'name': 'Aluminium blister pack'}, {'supercategory': 'Blister pack', 'id': 3, 'name': 'Carded blister pack'}, {'supercategory': 'Bottle', 'id': 4, 'name': 'Other plastic bottle'}, {'supercategory': 'Bottle', 'id': 5, 'name': 'Clear plastic bottle'}, {'supercategory': 'Bottle', 'id': 6, 'name': 'Glass bottle'}, {'supercategory': 'Bottle cap', 'id': 7, 'name': 'Plastic bottle cap'}, {'supercategory': 'Bottle cap', 'id': 8, 'name': 'Metal bottle cap'}, {'supercategory': 'Broken glass', 'id': 9, 'name': 'Broken glass'}, {'supercategory': 'Can', 'id': 10, 'name': 'Food Can'}, {'supercategory': 'Can', 'id': 11, 'name': 'Aerosol'}, {'supercategory': 'Can', 'id': 12, 'name': 'Drink can'}, {'supercategory': 'Carton', 'id': 13, 'name': 'Toilet tube'}, {'supercategory': 'Carton', 'id': 14, 'name': 'Other carton'}, {'supercate

In [8]:
print(coco_labels)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59}


On peut vérifier que l'on dipose de toutes les informations en affichant une dizaine de super catégories.

In [9]:
categories[:10]

[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'},
 {'supercategory': 'Battery', 'id': 1, 'name': 'Battery'},
 {'supercategory': 'Blister pack', 'id': 2, 'name': 'Aluminium blister pack'},
 {'supercategory': 'Blister pack', 'id': 3, 'name': 'Carded blister pack'},
 {'supercategory': 'Bottle', 'id': 4, 'name': 'Other plastic bottle'},
 {'supercategory': 'Bottle', 'id': 5, 'name': 'Clear plastic bottle'},
 {'supercategory': 'Bottle', 'id': 6, 'name': 'Glass bottle'},
 {'supercategory': 'Bottle cap', 'id': 7, 'name': 'Plastic bottle cap'},
 {'supercategory': 'Bottle cap', 'id': 8, 'name': 'Metal bottle cap'},
 {'supercategory': 'Broken glass', 'id': 9, 'name': 'Broken glass'}]

De même pour les catégories.

In [12]:
class_num

{'Aluminium foil': 0,
 'Battery': 1,
 'Aluminium blister pack': 2,
 'Carded blister pack': 3,
 'Other plastic bottle': 4,
 'Clear plastic bottle': 5,
 'Glass bottle': 6,
 'Plastic bottle cap': 7,
 'Metal bottle cap': 8,
 'Broken glass': 9,
 'Food Can': 10,
 'Aerosol': 11,
 'Drink can': 12,
 'Toilet tube': 13,
 'Other carton': 14,
 'Egg carton': 15,
 'Drink carton': 16,
 'Corrugated carton': 17,
 'Meal carton': 18,
 'Pizza box': 19,
 'Paper cup': 20,
 'Disposable plastic cup': 21,
 'Foam cup': 22,
 'Glass cup': 23,
 'Other plastic cup': 24,
 'Food waste': 25,
 'Glass jar': 26,
 'Plastic lid': 27,
 'Metal lid': 28,
 'Other plastic': 29,
 'Magazine paper': 30,
 'Tissues': 31,
 'Wrapping paper': 32,
 'Normal paper': 33,
 'Paper bag': 34,
 'Plastified paper bag': 35,
 'Plastic film': 36,
 'Six pack rings': 37,
 'Garbage bag': 38,
 'Other plastic wrapper': 39,
 'Single-use carrier bag': 40,
 'Polypropylene bag': 41,
 'Crisp packet': 42,
 'Spread tub': 43,
 'Tupperware': 44,
 'Disposable food

Si tout est bon, nous pouvons alors construire les fichiers texte.

### ➡️ À toi de jouer

Pour chaque image, nous allons créer un fichier texte qui contient les annotations. Pour cela, nous allons donc utiliser la variable `coco_labels_inverse` pour chaque ligne d'annotation, et remplir le reste de la ligne avec la bounding box associée.

In [ ]:
import numpy as np
import tqdm
import os
import shutil

In [ ]:
# Nous créons un dossier temporaire pour y mettre les images et les fichiers texte
!mkdir -p tmp/labels tmp/images

save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'
i = 0
# Cette boucle permet d'extraire le contenu de chaque fichier texte à partir du fichier annotation et de créer le fichier
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc="Conversion fichier JSON en fichier texte"):
    # Pour chaque image on extrait le nom et ses dimensions
    img_info = data_source.loadImgs(img_id)[0]
    i = i+1
    save_name = img_info['file_name'].replace('/', '_')
    file_name = save_name.split('.')[0]
    height = img_info['height']
    width = img_info['width']
    # On créé le fichier texte
    save_path = save_base_path + file_name + '.txt'
    is_exist = False #os.path.isfile(save_path)
    
    # On remplit le fichier
    with open(save_path, mode='w') as fp:
        # Extraction de l'identifiant de la catégorie
        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0: 
            fp.write('')
            continue
           
        # Récupération des bounding boxes
        annotations = data_source.loadAnns(annotation_id)
        lines = ''
        file_lines = []  
        for annotation in annotations:
          
          # Pour chaque annotation, récupérer le label depuis coco_labels_inverse et ajouter les bounding box sur la ligne.
          # TODO
          label = coco_labels_inverse[annotation['category_id']]
          is_exist = True
          """
          One row per object
          Each row is class x_center y_center width height format.
          Box coordinates must be in normalized xywh format (from 0 - 1).
          Class numbers are zero-indexed (start from 0).
          """
          bbox = annotation["bbox"]
          bbox[0] = round((bbox[0] + bbox[2] / 2) / width, 6)
          bbox[1] = round((bbox[1] + bbox[3] / 2) / height, 6)
          bbox[2] = round(bbox[2] / width, 6)
          bbox[3] = round(bbox[3] / height, 6)
          lines = str(str(label) +" "+ ' '.join(str(b) for b in bbox))
          fp.writelines(lines + '\n')
        
    # Si tout est OK, on enregistre dans le dossier data
    if  is_exist:
        shutil.copy('./tacotrashdataset/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
        #print("none")
        os.remove(save_path)

Conversion fichier JSON en fichier texte: 1500it [00:15, 94.75it/s] 


Pour terminer, il ne reste plus qu'à découper le dossier en trois sous-dossiers, qui vont représenter nos sous-ensembles d'entraînement, de test et de validation.

In [ ]:
import splitfolders

splitfolders.ratio("tmp", output="taco", ratio=(0.8, 0.1,0.1))

Copying files: 3000 files [00:34, 86.37 files/s]  


Au final, notre dossier `taco/` va contenir nos trois dossiers : nous pouvons alors supprimer le dossier temporaire `tmp/`.

In [ ]:
!rm -rf ./tmp